In [ ]:
!pip install yfinance
import pandas as pd
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use(seaborn)
import warnings
warnings.filterwarnings(ignore) 
# from IPython.display import FileLink
from google.colab import files
import tqdm.notebook as tq
from pandas_datareader import data as pdr
yf.pdr_override() # <== thats all it takes :-)
from datetime import date
import pandas as pd
import matplotlib.pyplot as plt

     |████████████████████████████████| 6.3 MB 35.8 MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.63-py2.py3-none-any.whl size=23919 sha256=7139487e2e3eb532da3c324d052605c2606a18f59a97911757e7fd0c588c27ac
  Stored in directory: /root/.cache/pip/wheels/fe/87/8b/7ec24486e001d3926537f5f7801f57a74d181be25b11157983
Successfully built yfinance
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
# # from tqdm import tqdm
# from google.colab import drive
# drive.mount("/content/gdrive")

In [ ]:
url = https://anaconda.org/conda-forge/libta-lib/0.4.0/download/linux-64/libta-lib-0.4.0-h516909a_0.tar.bz2
!curl -L $url | tar xj -C /usr/lib/x86_64-linux-gnu/ lib --strip-components=1
url = https://anaconda.org/conda-forge/ta-lib/0.4.19/download/linux-64/ta-lib-0.4.19-py37ha21ca33_2.tar.bz2
!curl -L $url | tar xj -C /usr/local/lib/python3.7/dist-packages/ lib/python3.7/site-packages/talib --strip-components=3


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3701    0  3701    0     0  32182      0 --:--:-- --:--:-- --:--:-- 32182
100  503k  100  503k    0     0  1884k      0 --:--:-- --:--:-- --:--:-- 1884k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3689    0  3689    0     0  39244      0 --:--:-- --:--:-- --:--:-- 39244
100  406k  100  406k    0     0  1767k      0 --:--:-- --:--:-- --:--:-- 1767k


In [ ]:
import talib

# Insights

Remove vwap condition 

**Insight** - During intraday vwap can be used to get target. If there is momentum price always be above vwap. Can exit if vwap is touched. ( get vwap parameters)

<!-- More weightage to stocks if stock comes under top 10 gainer or losser. -->

Extra confirmation if the volume is high

Select stocks which are near to 52 weeks high. ( 75% crossed, 25% pending to touch)

<!-- Check related index trend (should be performing good) and now check the stock performance (should be underperforming) means the stock will start growing following its index. -->


Rules to select stocks:

Only long calls logic no shorting. Will Write short logic later.
1.   Stock selection: Should be having reversal.
2.   RSI should be above 60
1.   Price should be above 20days moving average.
1.   Price should be above VWAP
1.   Price should +ve and Open Intrest should be +ve





Rules to Trade on selected stocks:

Time Frame 5min

1.   Open=Low (candle should not have lower tail) - 1st candle
2.   Volume should be High. - 1st candle
3.   2nd candle check manually should be green.
4.   Enter in 2nd candle High (Market Order). Keep SL day low.
5.   Should form new high in 1st candle itself.
6.   Target 1.5% or 3.15PM.



In [ ]:
nse_Symbol=[ADANIPORTS,
 ASIANPAINT,
 AXISBANK,
 BAJAJ-AUTO,
 BAJAJFINSV,
 BAJFINANCE,
 BHARTIARTL,
 BPCL,
 BRITANNIA,
 CIPLA,
 COALINDIA,
 DIVISLAB,
 DRREDDY,
 EICHERMOT,
 GRASIM,
 HCLTECH,
 HDFC,
 HDFCBANK,
 HDFCLIFE,
 HEROMOTOCO,
 HINDALCO,
 HINDUNILVR,
 ICICIBANK,
 INDUSINDBK,
 INFY,
 IOC,
 ITC,
 JSWSTEEL,
 KOTAKBANK,
 LT,
 M&M,
 MARUTI,
 NESTLEIND,
 NTPC,
 ONGC,
 POWERGRID,
 RELIANCE,
 SBILIFE,
 SBIN,
 SHREECEM,
 SUNPHARMA,
 TATACONSUM,
 TATAMOTORS,
 TATASTEEL,
 TCS,
 TECHM,
 TITAN,
 ULTRACEMCO,
 UPL,
 WIPRO]

In [ ]:
# # stock="ACC"
# start=date(2021,5,1)
# end=date(2021,7,1)
# # expiry_date=date(2021,7,29)

In [ ]:
def checker(stock_name,start_date,end_date):
    # get historic data
    df1 = yf.download(stock_name, 
                  start=start_date, 
                  end=end_date)
    # df1 = get_history(symbol=stock,start=start,end=end)
    df1[SMA]=talib.SMA(df1[Close],timeperiod=20) #SMA 
    df1[RSI] = talib.RSI(df1[Close], timeperiod=14) #RSI
    df1[price] =df1[Close].diff() # Price difference
    df1=df1.reset_index()
    data=df1.copy()
    #criteria for reversals
    data[Color] = [Red if data[Open][i] > data[Close][i] else Green if data[Open][i] < data[Close][i] else Neutral for i in range(0,len(data))]
    data[Downshadow] = [data[Open][i] - data[Low][i] if data[Color][i] == Green else data[Close][i] - data[Low][i] if data[Color][i] == Red else 0 for i in range(0,len(data))]
    data[Upshadow] = [data[High][i] - data[Close][i] if data[Color][i] == Green else data[High][i] - data[Open][i] if data[Color][i] == Red else 0 for i in range(0,len(data))]
    data[Body] =[abs(data[Close][i] - data[Open][i]) for i in range(0,len(data))]
    data[Pre Color]=data[Color].shift(1)
    data[Post Color]=data[Color].shift(-1)
    data[Pre Downshadow]=data[Downshadow].shift(1)
    data[Pre Upshadow]=data[Upshadow].shift(1)
    data[Pre Body]=data[Body].shift(1)
    data[Pre Open]=data[Open].shift(1)
    data[Pre Close]=data[Close].shift(1)
    data[Pre Low]=data[Low].shift(1)
    data[Pre High]=data[High].shift(1)
    data[Points] = abs(data[Open] - data[Close]).shift(-1)
    # reversal decisions
    reversal=[]
    for i in range(0,len(data)):
#         if data[Upshadow][i] > 2* data[Body][i] and data[Upshadow][i] > 4*data[Downshadow][i]:
#             reversal.append(Inverted Hammer)
        if data[Downshadow][i] > 2* data[Body][i] and data[Downshadow][i] > 4*data[Upshadow][i]:
            reversal.append(Hammer)
        elif data[Pre Open][i] > data[Pre Close][i] and data[Open][i] < data[Close][i] and  data[Pre Close][i] >= data[Open][i] and data[Pre Open][i] < data[Close][i]:
            reversal.append(Bull Enf)
        elif data[Pre Open][i] < data[Pre Close][i] and data[Open][i] > data[Close][i] and  data[Pre Close][i] <= data[Open][i] and data[Pre Open][i] > data[Close][i]:
            reversal.append(Bear Enf)
        elif data[Pre Open][i] > data[Pre Close][i] and data[Open][i] < data[Close][i] and  data[Pre Close][i] <= data[Open][i] and data[Pre Open][i] > data[Close][i]:
            reversal.append(Bull Har)
        elif data[Pre Open][i] < data[Pre Close][i] and data[Open][i] > data[Close][i] and  data[Pre Close][i] >= data[Open][i] and data[Pre Open][i] < data[Close][i]:
            reversal.append(Bear Har)
        elif data[Pre Body][i] < 1.5*data[Body][i]  and data[Body][i] < 1.5* data[Pre Body][i]  and data[Pre Body][i] > data[Pre Upshadow][i] and data[Pre Body][i] > data[Pre Downshadow][i] and  data[Body][i] > data[Upshadow][i] and data[Body][i] > data[Downshadow][i] and data[Pre Open][i] > data[Pre Close][i] and data[Open][i] < data[Close][i] and data[Pre Close][i] < data[Open][i] and data[Pre Open][i] < data[Close][i]:
            reversal.append(Bull Sash)
        elif data[Pre Body][i] < 1.5*data[Body][i]  and data[Body][i] < 1.5* data[Pre Body][i]  and data[Pre Body][i] > data[Pre Upshadow][i] and data[Pre Body][i] > data[Pre Downshadow][i] and  data[Body][i] > data[Upshadow][i] and data[Body][i] > data[Downshadow][i] and data[Pre Open][i] < data[Pre Close][i] and data[Open][i] > data[Close][i] and data[Pre Close][i] > data[Open][i] and data[Pre Open][i] > data[Close][i]:
            reversal.append(Bear Sash)
        elif data[Pre Body][i] < 1.5*data[Body][i]  and data[Body][i] < 1.5* data[Pre Body][i]  and data[Pre Body][i] > data[Pre Upshadow][i] and data[Pre Body][i] > data[Pre Downshadow][i] and  data[Body][i] > data[Upshadow][i] and data[Body][i] > data[Downshadow][i] and data[Pre Open][i] > data[Pre Close][i] and data[Open][i] < data[Close][i] and data[Pre Close][i] > data[Open][i] and data[Pre Open][i] > data[Close][i]:
            reversal.append(Piercing)
        elif data[Pre Body][i] < 1.5*data[Body][i]  and data[Body][i] < 1.5* data[Pre Body][i]  and data[Pre Body][i] > data[Pre Upshadow][i] and data[Pre Body][i] > data[Pre Downshadow][i] and  data[Body][i] > data[Upshadow][i] and data[Body][i] > data[Downshadow][i] and data[Pre Open][i] < data[Pre Close][i] and data[Open][i] > data[Close][i] and data[Pre Close][i] < data[Open][i] and data[Pre Open][i] < data[Close][i]:
            reversal.append(Dark Cloud)
        else:
            reversal.append(No)
    data[reversal]=reversal
    # add open interset information
    # data_fut = get_history(symbol=stock,futures=True,start=start, end=end,
    # expiry_date=expiry_date) 
    # data_fut=data_fut.reset_index()
    # data[Change in OI]=data_fut[Change in OI]
    # drop rest columns and drop based on need rules
    names=[Color, Downshadow, Upshadow,
           Body, Pre Color, Pre Downshadow, Pre Upshadow, Pre Body,
           Pre Open, Pre Close, Pre Low, Pre High,]
    data=data.drop(names,axis=1)
    data=data.dropna().reset_index(drop=True)
    data=data[data[Close] > data[SMA]]
    data=data[data[RSI] > 60]
    # data=data[data[Close] > data[VWAP]]
#     data=data[data[Change in OI] > 0]
    data=data[data[price] > 0]
    data=data.reset_index(drop=True)
    data=data[data[reversal].str.contains(Bull|Hammer|Piercing)].reset_index(drop=True)
    return (list(data[Close]),list(data[Date]),list(data[reversal]),list(data[Post Color]),list(data[Points]))

# create df of nested info per stock
selected_stocks={}
for i in tq.tqdm(nse_Symbol):
    selected_stocks[i]=checker(i,2021-04-01,2021-06-01)
final_selection=pd.DataFrame.from_dict(selected_stocks,orient=index,columns=[Close,Dates,Reversal,Color,Points])
final_selection.reset_index(inplace=True)
final_selection=final_selection.rename(columns={index:Ticker})
final_selection=final_selection.reset_index().drop(index,axis=1)
# one row
# add all in proper list format
new_col=[]
for i in tq.tqdm(range(0,len(final_selection))):
  new_entry=[]
  for j in range(0,len(final_selection[Dates][i])):
    new_entry.append([final_selection[Ticker][i],final_selection[Dates][i][j],final_selection[Close][i][j],final_selection[Reversal][i][j],final_selection[Color][i][j],final_selection[Points][i][j]])
  new_col.append(new_entry)
# make it to proper csv file
final = pd.DataFrame(columns=[Stock,Date,Close,Reversal,Color,Points])
for i in tq.tqdm(range(0,len(new_col))):
  for j in new_col[i]:
    try:
      final = final.append({Stock: j[0],Date:j[1],Close:j[2],Reversal:j[3],Color:j[4],Points:j[5]}, ignore_index=True)
    except:
      pass
# print(completed, please check dataframe "final" )
final=final.dropna().reset_index(drop=True)
# final[Date]=final[Date].astype(str)
# final[Shares]=  round(65500/final[Close] )
# final=final.sort_values(Date).reset_index(drop=True)
# final1=final[final[Color] == Red]
# final2=final[final[Color] == Green]
# Total=sum(final1[Shares] * final1[Points]) - len(final1) * 75
# Loss=sum(final2[Shares] * final2[Points]) + len(final2) * 75


[*********************100%***********************]  1 of 1 completed


NameError: ignored

In [ ]:
# print(Total,Total)
# print(Loss,Loss)

In [ ]:
final

,Stock,Date,Close,Reversal,Color,Points
0,ASIANPAINT.NS,2021-05-26,2941.600098,Hammer,Red,0.649902
1,BAJAJ-AUTO.NS,2021-05-26,4166.299805,Bull Enf,Green,76.100098
2,BAJAJFINSV.NS,2021-05-05,11114.450195,Bull Har,Red,72.049805
3,BAJAJFINSV.NS,2021-05-17,11249.349609,Bull Enf,Green,51.149414
4,BHARTIARTL.NS,2021-05-05,559.950012,Bull Har,Red,1.200012
5,COALINDIA.NS,2021-05-17,148.149994,Bull Har,Red,2.650009
6,COALINDIA.NS,2021-05-19,151.850006,Bull Enf,Red,5.750000
7,DIVISLAB.NS,2021-05-05,4035.449951,Bull Har,Green,13.050049
8,DIVISLAB.NS,2021-05-19,4039.649902,Bull Har,Red,8.849854
9,DIVISLAB.NS,2021-05-20,4040.800049,Hammer,Green,38.099854
